<a href="https://colab.research.google.com/github/baizhankyzy/academic-knowledge-base/blob/main/knowledge_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#  Step 1: Replace with your GitHub repo URL
GITHUB_REPO = "https://github.com/baizhankyzy/academic-knowledge-base.git"

#  Step 2: Remove old version if already cloned
!rm -rf academic-knowledge-base

#  Step 3: Clone your GitHub repo
!git clone {GITHUB_REPO}

#  Step 4: Set correct paths for file access
pdf_path = "academic-knowledge-base/data/ai-and-hmi/anticipatory-design.pdf"
output_path = "academic-knowledge-base/output/anticipatory-design.json"


Cloning into 'academic-knowledge-base'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 45 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 2.32 MiB | 7.95 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
# 📘 Knowledge Extraction from Academic PDFs to JSON (Google Colab-ready)

# ✅ Section A: Install & Import Packages
!pip install pdfplumber spacy yake -q
import os
import json
import pdfplumber
import spacy
import yake



# ✅ Section C: Load NLP Models
nlp = spacy.load("en_core_web_sm")
kw_extractor = yake.KeywordExtractor()

# ✅ Section D: Extract and Process Metadata from One PDF
def extract_metadata(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

    doc = nlp(text)
    title = text.split("\n")[0][:200]  # First line as fallback title
    keywords = [kw[0] for kw in kw_extractor.extract_keywords(text)[:10]]
    entities = list(set([ent.text for ent in doc.ents]))

    return {
        "title": title,
        "keywords": keywords,
        "entities": entities,
        "full_text": text,
        "file_name": os.path.basename(pdf_path)
    }

# ✅ Section E: Run on Single File and Save as JSON
data = extract_metadata(pdf_path)

with open(output_path, "w") as f:
    json.dump(data, f, indent=4)

print(f"✅ Extracted metadata from {pdf_path} and saved to {output_path}")


✅ Extracted metadata from academic-knowledge-base/data/ai-and-hmi/anticipatory-design.pdf and saved to academic-knowledge-base/output/anticipatory-design.json


## Upload single PDF

In [ ]:
import os
print(os.path.exists("data/ai-and-hmi/anticipatory-design.pdf"))  # Should return True


False


## Extract Raw text with pdf plumber

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])


FileNotFoundError: [Errno 2] No such file or directory: 'data/ai-and-hmi/anticipatory-design.pdf'

## Prerequisites

In [1]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install faiss-gpu
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

## Data ingestion

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#Load the document
loader = PyPDFLoader("academic-knowledge-base/data/ai-and-hmi/anticipatory-design.pdf")
documents = loader.load()

#Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)


## Data embedding and Storage with FAISS

In [6]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS

#Load embedding model
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)

#Create FAISS vectorstore
vectorstore = FAISS.from_documents(docs, embeddings)

#Save and reload the vector store
vectorstore.save_local("faiss_index")
persisted_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

#Create a retriever
retriever = persisted_vectorstore.as_retriever()

<ipython-input-6-e4f5113961dd>:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx